In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import sys
sys.path.append('../code/');

import simulate_data as sd
import encoding as enc

seed = 888;
np.random.seed(seed) # fix randomness

In [ ]:
deconv = sio.loadmat("../data/ADN29081-Z_20_10_2018/s_0_deconv.mat");
Y_real = (deconv['s']).T;
hd = sio.loadmat("../data/ADN29081-Z_20_10_2018/HD_180_deg_ADN29081-Z_20_10_2018.mat")['HD_180_deg_total'];
frame_map = sio.loadmat("../data/ADN29081-Z_20_10_2018/frameMap_ADN29081-Z_20_10_2018.mat")["frameMap"]
hd = hd[0,frame_map[:,0]-1];

bin_len=181;

In [ ]:
tuning_curve = enc.get_tc(Y_real, hd, bin_len);

# simulate from poisson distribution

In [ ]:
stims = np.repeat(hd, repeats=33);
stims = np.tile(stims,3);

In [ ]:
np.random.seed(seed);
factors = np.random.uniform(0.5,10,Y_real.shape[1]);

In [ ]:
spike_s = np.zeros((int(stims.shape[0]/33),Y_real.shape[1]));
cal = np.zeros((int(stims.shape[0]/33),Y_real.shape[1]));
noisy_cal = np.zeros((int(stims.shape[0]/33),Y_real.shape[1]));
for ii in range(Y_real.shape[1]):
    spike_s[:,ii], cal[:,ii], noisy_cal[:,ii] = sd.simfp(ii, tuning_curve*100, deconv['sn']*100, stims)

In [ ]:
spike_s = spike_s*np.array([factors]);
cal = cal*np.array([factors]);
noisy_cal = noisy_cal*np.array([factors]);

In [ ]:
sio.savemat("../data/simulated/sim_3stim_vary_snr.mat", 
           {'cal':cal, 'spikes':spike_s, 'noisy_cal':noisy_cal, 'factors':factors});

# simulate from negative binomial distribution

In [ ]:
stims = hd.copy();
stims = np.tile(stims,3);

In [ ]:
np.random.seed(seed);
factors = np.random.uniform(2,5,Y_real.shape[1]);

np.random.seed(seed);
factors2 = np.random.uniform(0.5,10,Y_real.shape[1]);

In [ ]:
spike_s = np.zeros((stims.shape[0],Y_real.shape[1]));
cal = np.zeros((stims.shape[0],Y_real.shape[1]));
noisy_cal = np.zeros((stims.shape[0],Y_real.shape[1]));
for ii in range(Y_real.shape[1]):
    spike_s[:,ii], cal[:,ii], noisy_cal[:,ii] = sd.simfnb(ii, tuning_curve*100, 
                                                    deconv['sn']*100, stims, factors=factors)

In [ ]:
spike_s = spike_s*np.array([factors2]);
cal = cal*np.array([factors2]);
noisy_cal = noisy_cal*np.array([factors2]);

In [ ]:
sio.savemat("../data/simulated/sim_3stim_vnb_vary_snr.mat", 
           {'cal':cal, 'spikes':spike_s, 'noisy_cal':noisy_cal, 'factors':factors,'factors2':factors2});